In [9]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import requests
import time
import csv
import json
from sqlalchemy import create_engine

## API call

In [10]:
client = Socrata("data.melbourne.vic.gov.au", None)
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("4n3a-s6rn", limit=50000)
# Convert to pandas DataFrame
historic_df = pd.DataFrame.from_records(results)
historic_df

,deviceid,arrivaltime,departuretime,durationminutes,streetmarker,signplateid,sign,areaname,streetid,streetname,betweenstreet1id,betweenstreet1,betweenstreet2id,betweenstreet2,sideofstreet,sidename,bayid,inviolation,vehiclepresent
0,17571,2020-02-26T10:47:05.000,2020-02-26T10:47:42.000,0,13720E,196,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6503,0,0
1,17571,2020-02-26T18:30:00.000,2020-02-27T00:00:00.000,330,13720E,NaN,NaN,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6503,0,1
2,17657,2020-02-12T18:06:46.000,2020-02-12T18:30:00.000,24,13823S,463,3P MTR M-SAT 7.30 - 18.30,Docklands,590,DOCKLANDS DRIVE,1097,PEARL RIVER ROAD,1405,WATTLE ROAD,4,South,6468,0,0
3,17580,2020-03-04T11:38:00.000,2020-03-04T11:38:12.000,0,13730E,196,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6508,0,1
4,17579,2020-02-24T07:27:01.000,2020-02-24T07:30:00.000,3,13728E,NaN,NaN,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6507,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,17642,2020-01-16T20:09:21.000,2020-01-16T20:37:41.000,28,13808N,NaN,NaN,Docklands,590,DOCKLANDS DRIVE,1097,PEARL RIVER ROAD,1398,WATERFRONT WAY,3,North,6474,0,0
49996,17617,2020-02-27T14:53:09.000,2020-02-27T16:46:35.000,113,13775W,196,2P MTR M-SAT 7:30-18:30,Docklands,1398,WATERFRONT WAY,915,LITTLE DOCKLANDS DRIVE,674,FOOTSCRAY ROAD,5,West,6557,0,1
49997,17579,2020-03-19T20:31:30.000,2020-03-19T20:31:48.000,0,13728E,NaN,NaN,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6507,0,1
49998,17586,2020-02-14T17:54:53.000,2020-02-14T17:56:24.000,2,13742E,196,2P MTR M-SAT 7:30-18:30,Docklands,1097,PEARL RIVER ROAD,590,DOCKLANDS DRIVE,1066,OBSERVATION DRIVE,2,East,6514,0,0


## Cleaning dataframes to join 

In [11]:
historic_df = historic_df[["bayid", "arrivaltime", "departuretime", "durationminutes", "areaname"]]
historic_df

,bayid,arrivaltime,departuretime,durationminutes,areaname
0,6503,2020-02-26T10:47:05.000,2020-02-26T10:47:42.000,0,Docklands
1,6503,2020-02-26T18:30:00.000,2020-02-27T00:00:00.000,330,Docklands
2,6468,2020-02-12T18:06:46.000,2020-02-12T18:30:00.000,24,Docklands
3,6508,2020-03-04T11:38:00.000,2020-03-04T11:38:12.000,0,Docklands
4,6507,2020-02-24T07:27:01.000,2020-02-24T07:30:00.000,3,Docklands
...,...,...,...,...,...
49995,6474,2020-01-16T20:09:21.000,2020-01-16T20:37:41.000,28,Docklands
49996,6557,2020-02-27T14:53:09.000,2020-02-27T16:46:35.000,113,Docklands
49997,6507,2020-03-19T20:31:30.000,2020-03-19T20:31:48.000,0,Docklands
49998,6514,2020-02-14T17:54:53.000,2020-02-14T17:56:24.000,2,Docklands


In [12]:
# historic_df['bayid'] = historic_df['bayid'].astype(int)

In [13]:
historic_df.dtypes

bayid              object
arrivaltime        object
departuretime      object
durationminutes    object
areaname           object
dtype: object

In [14]:
historic_df.rename(columns = {'bayid':'bay_id'}, inplace = True) 
historic_df

/Users/yasminekhalifa/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,bay_id,arrivaltime,departuretime,durationminutes,areaname
0,6503,2020-02-26T10:47:05.000,2020-02-26T10:47:42.000,0,Docklands
1,6503,2020-02-26T18:30:00.000,2020-02-27T00:00:00.000,330,Docklands
2,6468,2020-02-12T18:06:46.000,2020-02-12T18:30:00.000,24,Docklands
3,6508,2020-03-04T11:38:00.000,2020-03-04T11:38:12.000,0,Docklands
4,6507,2020-02-24T07:27:01.000,2020-02-24T07:30:00.000,3,Docklands
...,...,...,...,...,...
49995,6474,2020-01-16T20:09:21.000,2020-01-16T20:37:41.000,28,Docklands
49996,6557,2020-02-27T14:53:09.000,2020-02-27T16:46:35.000,113,Docklands
49997,6507,2020-03-19T20:31:30.000,2020-03-19T20:31:48.000,0,Docklands
49998,6514,2020-02-14T17:54:53.000,2020-02-14T17:56:24.000,2,Docklands


In [15]:
historic_df = historic_df.dropna(subset = ["areaname"])
historic_df

,bay_id,arrivaltime,departuretime,durationminutes,areaname
0,6503,2020-02-26T10:47:05.000,2020-02-26T10:47:42.000,0,Docklands
1,6503,2020-02-26T18:30:00.000,2020-02-27T00:00:00.000,330,Docklands
2,6468,2020-02-12T18:06:46.000,2020-02-12T18:30:00.000,24,Docklands
3,6508,2020-03-04T11:38:00.000,2020-03-04T11:38:12.000,0,Docklands
4,6507,2020-02-24T07:27:01.000,2020-02-24T07:30:00.000,3,Docklands
...,...,...,...,...,...
49995,6474,2020-01-16T20:09:21.000,2020-01-16T20:37:41.000,28,Docklands
49996,6557,2020-02-27T14:53:09.000,2020-02-27T16:46:35.000,113,Docklands
49997,6507,2020-03-19T20:31:30.000,2020-03-19T20:31:48.000,0,Docklands
49998,6514,2020-02-14T17:54:53.000,2020-02-14T17:56:24.000,2,Docklands


In [16]:
historic_df.drop_duplicates("bay_id", inplace=True)
historic_df

,bay_id,arrivaltime,departuretime,durationminutes,areaname
0,6503,2020-02-26T10:47:05.000,2020-02-26T10:47:42.000,0,Docklands
2,6468,2020-02-12T18:06:46.000,2020-02-12T18:30:00.000,24,Docklands
3,6508,2020-03-04T11:38:00.000,2020-03-04T11:38:12.000,0,Docklands
4,6507,2020-02-24T07:27:01.000,2020-02-24T07:30:00.000,3,Docklands
5,6471,2020-02-09T14:50:38.000,2020-02-09T16:08:38.000,78,Docklands
...,...,...,...,...,...
46304,5346,2020-04-13T23:00:00.000,2020-04-14T00:00:00.000,60,Southbank
48496,5323,2020-02-01T14:18:13.000,2020-02-01T14:18:41.000,0,Southbank
49237,6838,2020-01-31T16:37:56.000,2020-01-31T17:52:20.000,75,University
49340,6874,2020-02-14T06:49:07.000,2020-02-14T07:30:00.000,41,University


## API call

In [17]:
client = Socrata("data.melbourne.vic.gov.au", None)
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("vh2v-4nfs", limit=50000)
# Convert to pandas DataFrame
live_df = pd.DataFrame.from_records(results)
live_df

,bay_id,st_marker_id,status,location,lat,lon,:@computed_region_evbi_jbp8
0,6150,C13704,Unoccupied,"{'latitude': '-37.820019978880744', 'longitude...",-37.820019978880744,144.94315996150792,1
1,5486,11895W,Unoccupied,"{'latitude': '-37.81033788188465', 'longitude'...",-37.81033788188465,144.97811715353254,1
2,8920,18026E,Unoccupied,"{'latitude': '-37.80275309310207', 'longitude'...",-37.80275309310207,144.95976223025588,1
3,6358,13518E,Unoccupied,"{'latitude': '-37.821566925397846', 'longitude...",-37.821566925397846,144.95044111665504,1
4,4048,C6134,Unoccupied,"{'latitude': '-37.80434049684882', 'longitude'...",-37.80434049684882,144.952406825475,1
...,...,...,...,...,...,...,...
3169,5580,12093W,Unoccupied,"{'latitude': '-37.81083727337871', 'longitude'...",-37.81083727337871,144.98335547111526,1
3170,3364,6171W,Unoccupied,"{'latitude': '-37.80426415060094', 'longitude'...",-37.80426415060094,144.94314065249569,1
3171,6390,13636E,Unoccupied,"{'latitude': '-37.82036873067143', 'longitude'...",-37.82036873067143,144.94457489244326,1
3172,3007,5068E,Unoccupied,"{'latitude': '-37.805044328852205', 'longitude...",-37.805044328852205,144.95801660850978,1


In [18]:
live_df.drop_duplicates("bay_id", inplace=True)
live_df

,bay_id,st_marker_id,status,location,lat,lon,:@computed_region_evbi_jbp8
0,6150,C13704,Unoccupied,"{'latitude': '-37.820019978880744', 'longitude...",-37.820019978880744,144.94315996150792,1
1,5486,11895W,Unoccupied,"{'latitude': '-37.81033788188465', 'longitude'...",-37.81033788188465,144.97811715353254,1
2,8920,18026E,Unoccupied,"{'latitude': '-37.80275309310207', 'longitude'...",-37.80275309310207,144.95976223025588,1
3,6358,13518E,Unoccupied,"{'latitude': '-37.821566925397846', 'longitude...",-37.821566925397846,144.95044111665504,1
4,4048,C6134,Unoccupied,"{'latitude': '-37.80434049684882', 'longitude'...",-37.80434049684882,144.952406825475,1
...,...,...,...,...,...,...,...
3169,5580,12093W,Unoccupied,"{'latitude': '-37.81083727337871', 'longitude'...",-37.81083727337871,144.98335547111526,1
3170,3364,6171W,Unoccupied,"{'latitude': '-37.80426415060094', 'longitude'...",-37.80426415060094,144.94314065249569,1
3171,6390,13636E,Unoccupied,"{'latitude': '-37.82036873067143', 'longitude'...",-37.82036873067143,144.94457489244326,1
3172,3007,5068E,Unoccupied,"{'latitude': '-37.805044328852205', 'longitude...",-37.805044328852205,144.95801660850978,1


In [19]:
new_df = live_df[["bay_id", "lat", "lon"]]
new_df

,bay_id,lat,lon
0,6150,-37.820019978880744,144.94315996150792
1,5486,-37.81033788188465,144.97811715353254
2,8920,-37.80275309310207,144.95976223025588
3,6358,-37.821566925397846,144.95044111665504
4,4048,-37.80434049684882,144.952406825475
...,...,...,...
3169,5580,-37.81083727337871,144.98335547111526
3170,3364,-37.80426415060094,144.94314065249569
3171,6390,-37.82036873067143,144.94457489244326
3172,3007,-37.805044328852205,144.95801660850978


In [ ]:
# new_df['bay_id'] = new_df['bay_id'].astype(int)

## Merging dataframes 

In [20]:
df_merge_col = pd.merge(historic_df, new_df, how='right', on='bay_id')
df_merge_col

,bay_id,arrivaltime,departuretime,durationminutes,areaname,lat,lon
0,6419,2020-01-28T12:57:54.000,2020-01-28T14:09:56.000,72,Docklands,-37.82099788099109,144.94532421874825
1,5070,2020-04-05T02:00:00.000,2020-04-05T07:30:00.000,330,Southbank,-37.825375869666324,144.96959683137294
2,6428,2020-01-14T15:36:58.000,2020-01-14T15:38:34.000,2,Docklands,-37.82051140418841,144.94609471266074
3,6434,2020-01-03T10:37:06.000,2020-01-03T13:41:37.000,184,Docklands,-37.82014416727483,144.9459997769062
4,6429,2020-01-06T09:14:18.000,2020-01-06T11:40:01.000,146,Docklands,-37.82045109868588,144.94607848013453
...,...,...,...,...,...,...,...
3169,5580,NaN,NaN,NaN,NaN,-37.81083727337871,144.98335547111526
3170,3364,NaN,NaN,NaN,NaN,-37.80426415060094,144.94314065249569
3171,6390,NaN,NaN,NaN,NaN,-37.82036873067143,144.94457489244326
3172,3007,NaN,NaN,NaN,NaN,-37.805044328852205,144.95801660850978


In [21]:
df_merge_col.drop_duplicates("bay_id", inplace=True)
df_merge_col

,bay_id,arrivaltime,departuretime,durationminutes,areaname,lat,lon
0,6419,2020-01-28T12:57:54.000,2020-01-28T14:09:56.000,72,Docklands,-37.82099788099109,144.94532421874825
1,5070,2020-04-05T02:00:00.000,2020-04-05T07:30:00.000,330,Southbank,-37.825375869666324,144.96959683137294
2,6428,2020-01-14T15:36:58.000,2020-01-14T15:38:34.000,2,Docklands,-37.82051140418841,144.94609471266074
3,6434,2020-01-03T10:37:06.000,2020-01-03T13:41:37.000,184,Docklands,-37.82014416727483,144.9459997769062
4,6429,2020-01-06T09:14:18.000,2020-01-06T11:40:01.000,146,Docklands,-37.82045109868588,144.94607848013453
...,...,...,...,...,...,...,...
3169,5580,NaN,NaN,NaN,NaN,-37.81083727337871,144.98335547111526
3170,3364,NaN,NaN,NaN,NaN,-37.80426415060094,144.94314065249569
3171,6390,NaN,NaN,NaN,NaN,-37.82036873067143,144.94457489244326
3172,3007,NaN,NaN,NaN,NaN,-37.805044328852205,144.95801660850978


In [22]:
df_merge_col = df_merge_col.dropna()
df_merge_col

,bay_id,arrivaltime,departuretime,durationminutes,areaname,lat,lon
0,6419,2020-01-28T12:57:54.000,2020-01-28T14:09:56.000,72,Docklands,-37.82099788099109,144.94532421874825
1,5070,2020-04-05T02:00:00.000,2020-04-05T07:30:00.000,330,Southbank,-37.825375869666324,144.96959683137294
2,6428,2020-01-14T15:36:58.000,2020-01-14T15:38:34.000,2,Docklands,-37.82051140418841,144.94609471266074
3,6434,2020-01-03T10:37:06.000,2020-01-03T13:41:37.000,184,Docklands,-37.82014416727483,144.9459997769062
4,6429,2020-01-06T09:14:18.000,2020-01-06T11:40:01.000,146,Docklands,-37.82045109868588,144.94607848013453
...,...,...,...,...,...,...,...
103,5346,2020-04-13T23:00:00.000,2020-04-14T00:00:00.000,60,Southbank,-37.82539182057477,144.9605934088404
104,5323,2020-02-01T14:18:13.000,2020-02-01T14:18:41.000,0,Southbank,-37.827939243427664,144.97111569612792
105,6838,2020-01-31T16:37:56.000,2020-01-31T17:52:20.000,75,University,-37.79676019191005,144.95813214056
106,6874,2020-02-14T06:49:07.000,2020-02-14T07:30:00.000,41,University,-37.794426884822094,144.95841911683556


In [23]:
df_merge_col["areaname"] = df_merge_col["areaname"].str.lower()
df_merge_col

/Users/yasminekhalifa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,bay_id,arrivaltime,departuretime,durationminutes,areaname,lat,lon
0,6419,2020-01-28T12:57:54.000,2020-01-28T14:09:56.000,72,docklands,-37.82099788099109,144.94532421874825
1,5070,2020-04-05T02:00:00.000,2020-04-05T07:30:00.000,330,southbank,-37.825375869666324,144.96959683137294
2,6428,2020-01-14T15:36:58.000,2020-01-14T15:38:34.000,2,docklands,-37.82051140418841,144.94609471266074
3,6434,2020-01-03T10:37:06.000,2020-01-03T13:41:37.000,184,docklands,-37.82014416727483,144.9459997769062
4,6429,2020-01-06T09:14:18.000,2020-01-06T11:40:01.000,146,docklands,-37.82045109868588,144.94607848013453
...,...,...,...,...,...,...,...
103,5346,2020-04-13T23:00:00.000,2020-04-14T00:00:00.000,60,southbank,-37.82539182057477,144.9605934088404
104,5323,2020-02-01T14:18:13.000,2020-02-01T14:18:41.000,0,southbank,-37.827939243427664,144.97111569612792
105,6838,2020-01-31T16:37:56.000,2020-01-31T17:52:20.000,75,university,-37.79676019191005,144.95813214056
106,6874,2020-02-14T06:49:07.000,2020-02-14T07:30:00.000,41,university,-37.794426884822094,144.95841911683556


In [24]:
df_merge_col['bay_id'] = df_merge_col['bay_id'].astype(str)

/Users/yasminekhalifa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Normalising Dataframes 

In [25]:
# Converting columns in the dataframe to separate tables to be saved as csv 
def convert_column_to_fk(df, column, output_dir):
    values = sorted(list(df[column].unique()))
    ids = range(1, len(values) + 1)    
    fk_df = pd.DataFrame(values, index = ids).reset_index()
    fk_df.columns = ["id", column]
    fk_df.to_csv(f"{output_dir}/{column}.csv", index = False)
    df[column] = df[column].replace(dict(zip(values, ids)))
    df = df.rename(columns = {column: f"{column}_id"})
    return df

In [26]:
df_merge_col['areaname'] = df_merge_col['areaname'].astype(str)

/Users/yasminekhalifa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
complete = convert_column_to_fk(df_merge_col,"areaname","output")
complete

/Users/yasminekhalifa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,bay_id,arrivaltime,departuretime,durationminutes,areaname_id,lat,lon
0,6419,2020-01-28T12:57:54.000,2020-01-28T14:09:56.000,72,2,-37.82099788099109,144.94532421874825
1,5070,2020-04-05T02:00:00.000,2020-04-05T07:30:00.000,330,7,-37.825375869666324,144.96959683137294
2,6428,2020-01-14T15:36:58.000,2020-01-14T15:38:34.000,2,2,-37.82051140418841,144.94609471266074
3,6434,2020-01-03T10:37:06.000,2020-01-03T13:41:37.000,184,2,-37.82014416727483,144.9459997769062
4,6429,2020-01-06T09:14:18.000,2020-01-06T11:40:01.000,146,2,-37.82045109868588,144.94607848013453
...,...,...,...,...,...,...,...
103,5346,2020-04-13T23:00:00.000,2020-04-14T00:00:00.000,60,7,-37.82539182057477,144.9605934088404
104,5323,2020-02-01T14:18:13.000,2020-02-01T14:18:41.000,0,7,-37.827939243427664,144.97111569612792
105,6838,2020-01-31T16:37:56.000,2020-01-31T17:52:20.000,75,9,-37.79676019191005,144.95813214056
106,6874,2020-02-14T06:49:07.000,2020-02-14T07:30:00.000,41,9,-37.794426884822094,144.95841911683556


## Structuring tables for creating database

In [28]:
parking_bay = complete[["bay_id", "lat", "lon", "areaname_id"]]
# parking_bay.set_index("bay_id", inplace=True)
parking_bay

,bay_id,lat,lon,areaname_id
0,6419,-37.82099788099109,144.94532421874825,2
1,5070,-37.825375869666324,144.96959683137294,7
2,6428,-37.82051140418841,144.94609471266074,2
3,6434,-37.82014416727483,144.9459997769062,2
4,6429,-37.82045109868588,144.94607848013453,2
...,...,...,...,...
103,5346,-37.82539182057477,144.9605934088404,7
104,5323,-37.827939243427664,144.97111569612792,7
105,6838,-37.79676019191005,144.95813214056,9
106,6874,-37.794426884822094,144.95841911683556,9


In [29]:
parking_bay.dtypes

bay_id         object
lat            object
lon            object
areaname_id     int64
dtype: object

In [30]:
parking_duration = complete[["bay_id", "arrivaltime", "departuretime", "durationminutes"]]
# parking_duration.set_index("bay_id", inplace=True)
parking_duration

,bay_id,arrivaltime,departuretime,durationminutes
0,6419,2020-01-28T12:57:54.000,2020-01-28T14:09:56.000,72
1,5070,2020-04-05T02:00:00.000,2020-04-05T07:30:00.000,330
2,6428,2020-01-14T15:36:58.000,2020-01-14T15:38:34.000,2
3,6434,2020-01-03T10:37:06.000,2020-01-03T13:41:37.000,184
4,6429,2020-01-06T09:14:18.000,2020-01-06T11:40:01.000,146
...,...,...,...,...
103,5346,2020-04-13T23:00:00.000,2020-04-14T00:00:00.000,60
104,5323,2020-02-01T14:18:13.000,2020-02-01T14:18:41.000,0
105,6838,2020-01-31T16:37:56.000,2020-01-31T17:52:20.000,75
106,6874,2020-02-14T06:49:07.000,2020-02-14T07:30:00.000,41


In [31]:
parking_duration.dtypes


bay_id             object
arrivaltime        object
departuretime      object
durationminutes    object
dtype: object

## Create database connection

In [32]:
connection_string = "postgres:Yas.123@localhost:5432/Parking_db"
engine = create_engine(f'postgresql://{connection_string}')

In [33]:
engine.table_names()

['Area', 'Parking_bay', 'Parking_duration']

## Load DataFrames into database

In [34]:
parking_bay.to_sql(name='Parking_bay', con=engine, if_exists='append', index=False)

In [35]:
parking_duration.to_sql(name='Parking_duration', con=engine, if_exists='append', index=False)